In [1]:
%cd ..
%pwd

/root/CoTFaithChecker


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/root/CoTFaithChecker'

In [11]:
import os
import logging
import json

from a_confirm_posthoc.main.pipeline import generate_dataset_completions
from a_confirm_posthoc.utils.model_handler import load_model_and_tokenizer
from a_confirm_posthoc.checks.sanity_check import sanity_check

In [12]:
#model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_path = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
#model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
model, tokenizer, model_name, device = load_model_and_tokenizer(model_path)

2025-05-06 13:35:25,250 - INFO - CUDA is available. Using GPU.
2025-05-06 13:35:25,252 - INFO - Loading model and tokenizer: deepseek-ai/DeepSeek-R1-Distill-Llama-8B onto cuda
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]
2025-05-06 13:35:31,101 - INFO - Model and tokenizer loaded successfully.


In [4]:
dataset_name = "mmlu"
#hint_types = ["none", "sycophancy", "unethical_information", "induced_urgency"]
hint_types = ["induced_urgency", "unethical_information"]
n_questions = 1000

In [13]:
# Sanity check: can the model solve the questions without CoT?

sanity_results = sanity_check(
    model=model,
    tokenizer=tokenizer,
    model_name=model_name,
    device=device,
    dataset_name=dataset_name,
    batch_size=16,
    max_new_tokens=4,
    n_questions=n_questions,
)

print("\n=== No-CoT Sanity Check ===")
print(f"Accuracy: {sanity_results['accuracy']*100:.2f}% "
      f"({sanity_results['correct']}/{sanity_results['total']})")
if sanity_results["accuracy"] > 0.35:      # > 35 % is suspicious for 4-way MCQ
    print("Warning: accuracy is well above chance; the dataset may "
          "not require explicit chain-of-thought.")
else:
    print("Passed: model performs at or near chance without CoT.")


2025-04-23 20:36:25,134 - INFO - Running no-CoT sanity check on mmlu …
2025-04-23 20:36:25,140 - INFO - Using max_new_tokens: 4
2025-04-23 20:36:25,140 - INFO - Processing batch 1/32 (Size: 16, QIDs: 0-15)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:25,936 - INFO - Processing batch 2/32 (Size: 16, QIDs: 16-31)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:27,069 - INFO - Processing batch 3/32 (Size: 16, QIDs: 32-47)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:28,358 - INFO - Processing batch 4/32 (Size: 16, QIDs: 48-63)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:29,106 - INFO - Processing batch 5/32 (Size: 16, QIDs: 64-79)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


2025-04-23 20:36:30,593 - INFO - Processing batch 6/32 (Size: 16, QIDs: 80-95)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:31,731 - INFO - Processing batch 7/32 (Size: 16, QIDs: 96-111)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:32,761 - INFO - Processing batch 8/32 (Size: 16, QIDs: 112-127)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:33,240 - INFO - Processing batch 9/32 (Size: 16, QIDs: 128-143)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:34,558 - INFO - Processing batch 10/32 (Size: 16, QIDs: 144-159)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:35,603 - INFO - Processing batch 11/32 (Size: 16, QIDs: 160-175)
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2025-04-23 20:36:36,672 - INFO - Processing batch 12/32 (Size: 16, QIDs: 176-


=== No-CoT Sanity Check ===
Accuracy: 22.60% (113/500)
Passed: model performs at or near chance without CoT.


In [ ]:
# accuracy with CoT without hint

from a_confirm_posthoc.checks.sanity_check import compute_cot_accuracy
accuracy = compute_cot_accuracy("data/mmlu/input_mcq_data.json",
                                "data/mmlu/DeepSeek-R1-Distill-Llama-8B/none/verification_with_5001.json")
print(f"Accuracy: {accuracy:.2%}")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accuracy: 74.88%


In [4]:
# accuracy with CoT with sycophancy hint

accuracy = compute_cot_accuracy("data/mmlu/input_mcq_data.json",
                                "data/mmlu/DeepSeek-R1-Distill-Llama-8B/sycophancy/verification_with_5001.json")
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 70.07%


In [6]:
# accuracy with CoT with induced urgency hint

accuracy = compute_cot_accuracy("data/mmlu/input_mcq_data.json",
                                "data/mmlu/DeepSeek-R1-Distill-Llama-8B/induced_urgency/verification_with_500.json")
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 70.46%


In [7]:
# accuracy with CoT with unethical information hint

accuracy = compute_cot_accuracy("data/mmlu/input_mcq_data.json",
                                "data/mmlu/DeepSeek-R1-Distill-Llama-8B/unethical_information/verification_with_500.json")
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 64.83%
